<a href="https://colab.research.google.com/github/Gbell26/GestureBot/blob/main/Gesture_Recognizer_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.6 MB/s eta 0:00:00
  Created wheel for s

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
!ls

gesture_training.zip  sample_data


In [ ]:
!unzip gesture_training.zip
dataset_path = "gesture_training"

Archive:  gesture_training.zip
   creating: gesture_training/middle_thumb/
  inflating: gesture_training/middle_thumb/IMG_9392.jpg  
  inflating: gesture_training/middle_thumb/IMG_9393.jpg  
  inflating: gesture_training/middle_thumb/IMG_9394.jpg  
  inflating: gesture_training/middle_thumb/IMG_9395.jpg  
  inflating: gesture_training/middle_thumb/IMG_9396.jpg  
  inflating: gesture_training/middle_thumb/IMG_9397.jpg  
  inflating: gesture_training/middle_thumb/IMG_9398.jpg  
  inflating: gesture_training/middle_thumb/IMG_9399.jpg  
  inflating: gesture_training/middle_thumb/IMG_9400.jpg  
  inflating: gesture_training/middle_thumb/IMG_9401.jpg  
  inflating: gesture_training/middle_thumb/IMG_9402.jpg  
  inflating: gesture_training/middle_thumb/IMG_9403.jpg  
  inflating: gesture_training/middle_thumb/IMG_9404.jpg  
  inflating: gesture_training/middle_thumb/IMG_9405.jpg  
  inflating: gesture_training/middle_thumb/IMG_9406.jpg  
  inflating: gesture_training/middle_thumb/IMG_9407.jpg

In [ ]:
#load the data
#Roughly 45 images of e
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

gesture_training
['pointer_thumb', 'open_hand', 'none', 'pinky_thumb', 'ring_thumb', 'middle_thumb']


In [ ]:
#split the data into training, val, test data

data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
#Create Model

hparams = gesture_recognizer.HParams(export_dir="exported_model", epochs = 200, shuffle=True)
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_2 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 re_lu_2 (ReLU)              (None, 128)               0         
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 6)                 774       
 out (Dense)                                                     
                                                           

In [ ]:
#eval model
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

21/21 [==============================] - 0s 1ms/step - loss: 0.2692 - categorical_accuracy: 0.8571
Test loss:0.2692089378833771, Test accuracy:0.8571428656578064


In [ ]:
#export as tf lite model
model.export_model()
!ls exported_model

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
best_model_weights.data-00000-of-00001	checkpoint    gesture_recognizer.task  metadata.json
best_model_weights.index		epoch_models  logs


In [ ]:
#download new model
files.download('exported_model/gesture_recognizer.task')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>